## Segmenting and Clustering Neighboring in Toronto - Assignment - 1.3

In [1]:
# import libraries
import pandas as pd
import numpy as np
import requests
import geocoder
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

## 1. Download and Explore Dataset

In [2]:
# URL of the page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Download the data
page= requests.get(url).text

# dataframe creation
data = pd.read_html(page, header=0, attrs={"class":"wikitable sortable"})[0]

In [3]:
data.head()  # have a look at the dataframe

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
data.columns = ['Postcode', 'Borough', 'Neighborhood']

#### dimensions of data

In [5]:
data.shape                     # dimensions of data

(289, 3)

### Data Cleaning

In [6]:
data['Borough'].value_counts()           # to count Not assigned values

Not assigned        77
Etobicoke           45
North York          38
Scarborough         38
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [7]:
df = data[data['Borough']!='Not assigned']          # taking Borough with no "Not assignment"

In [8]:
df['Borough'].value_counts()            # checking unique values

Etobicoke           45
North York          38
Scarborough         38
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [9]:
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [10]:
df=df.reset_index(drop=True)                   # Reseting index to start from 0
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
df[df['Neighborhood']=='Not assigned']

,Postcode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [12]:
df['Neighborhood']= np.where(df['Neighborhood']=='Not assigned', df['Borough'], df['Neighborhood'])

### Data Aggregation / Transformation

In [13]:
df = df.groupby(['Postcode','Borough'], sort=False).agg(lambda x: ','.join(x))
df.reset_index(inplace=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Looking at the final shape of cleaned data

In [14]:
df.shape

(103, 3)

### Getting Coordinates for each neighborhood

In [15]:
geocodes = pd.read_csv('C:/Data/Geospatial_Coordinates.csv')  # Loading Coordinates from the provided file

In [16]:
geocodes.columns = ['Postcode', 'Latitude', 'Longitude']      # Setting Columns heading

In [17]:
geocodes.head()                # Exploring the latitudes and logitudes

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
dff =  pd.merge(df, geocodes, on='Postcode', how='inner')     # Joining the two dataframes to get the required result

In [19]:
dff.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### To Check Borroughs and Neighborhoods of Toronto

In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dff['Borough'].unique()),
        dff.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### To create map of Torono using latitude and longitude values

In [21]:
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('Toronto, Ontario')
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

latitude = 43.6532
longitude = -79.3832

print(latitude)
print(longitude)

43.6532
-79.3832


In [24]:
# address = 'Toronto, Ontario'
# geolocator = Nominatim()
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude

latitude = 43.65332
longitude = -79.38524

print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.65332, -79.38524.


In [25]:
import folium
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dff['Latitude'], dff['Longitude'], dff['Borough'], dff['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
# Show the map
toronto_map

In [26]:
CLIENT_ID = 'OWYE2T01OGZTG1LPOCYYRR22COTUSKK24IKOXWGHOBQZI45Q' # your Foursquare ID
CLIENT_SECRET = 'ZO4011O2MPXTIHHNFT0AII4EHY1CI0NDUQSKDBVM5BT3LQDY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OWYE2T01OGZTG1LPOCYYRR22COTUSKK24IKOXWGHOBQZI45Q
CLIENT_SECRET:ZO4011O2MPXTIHHNFT0AII4EHY1CI0NDUQSKDBVM5BT3LQDY


In [27]:
dff.loc[0, 'Neighborhood']

'Parkwoods'

In [28]:
neighborhood_latitude = dff.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dff.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dff.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OWYE2T01OGZTG1LPOCYYRR22COTUSKK24IKOXWGHOBQZI45Q&client_secret=ZO4011O2MPXTIHHNFT0AII4EHY1CI0NDUQSKDBVM5BT3LQDY&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c2f27044434b918453fde56'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'location': {'address': 'Toronto',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 245,
        'formattedAddress': ['Toronto', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'lat

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Parkwoods

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
parkwoods_venues = getNearbyVenues(names=dff['Neighborhood'],
                                   latitudes=dff['Latitude'],
                                   longitudes=dff['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [36]:
print(parkwoods_venues.shape)
parkwoods_venues.head()

(2229, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [37]:
parkwoods_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [38]:
print('There are {} uniques categories.'.format(len(parkwoods_venues['Venue Category'].unique())))

There are 272 uniques categories.


## 3. Analyze Each Neighborhood

In [39]:
# one hot encoding
parkwoods_onehot = pd.get_dummies(parkwoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
parkwoods_onehot['Neighborhood'] = parkwoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [parkwoods_onehot.columns[-1]] + list(parkwoods_onehot.columns[:-1])
parkwoods_onehot = parkwoods_onehot[fixed_columns]

parkwoods_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
parkwoods_onehot.shape

(2229, 272)

In [41]:
parkwoods_grouped = parkwoods_onehot.groupby('Neighborhood').mean().reset_index()
parkwoods_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
parkwoods_grouped.shape

(99, 272)

In [43]:
num_top_venues = 5

for hood in parkwoods_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = parkwoods_grouped[parkwoods_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
            venue  freq
0  Sandwich Place  0.25
1          Lounge  0.25
2    Skating Rink  0.25
3  Breakfast Spot  0.25
4     Yoga Studio  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground  0.33
1                     Bakery  0.33
2                       Park  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.22
1           Beer Store  0.11
2       Sandwich Place  0.11
3  Fried Chicken Joint  0.11
4          Coffee Shop  0.11


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place 

              venue  freq
0    Discount Store  0.29
1        Hobby Shop  0.14
2       Coffee Shop  0.14
3       Bus Station  0.14
4  Department Store  0.14


----East Toronto----
                       venue  freq
0                       Park   0.5
1          Convenience Store   0.5
2                Yoga Studio   0.0
3  Middle Eastern Restaurant   0.0
4              Movie Theater   0.0


----Emery,Humberlea----
                       venue  freq
0             Baseball Field   1.0
1  Middle Eastern Restaurant   0.0
2              Movie Theater   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.10
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.07
3         Women's Store  0.03
4                Bakery  0.03


----First Canadian Place,Underground city----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.08
2                Hotel 

                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.05
2                Café  0.04
3  Italian Restaurant  0.03
4               Hotel  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2               Bakery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05


----The Annex,North Midtown,Yorkville----
            venue  freq
0     Coffee Shop  0.12
1            Café  0.12
2  Sandwich Place  0.12
3     Pizza Place  0.08
4  History Museum  0.04


----The Beaches----
                        venue  freq
0                 Music Venue  0.25
1                 Coffee Shop  0.25
2                         Pub  0.25
3  Modern European Restaurant  0.00
4               Moving Target  0.00


----The Beaches West,India Bazaar----
              venue  freq
0              Park  0.10
1        Board Shop  0.05
2      Burger Joint  0.05
3    Sandwich Place  0.05
4  Sushi Restaurant  0.05


----The 

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = parkwoods_grouped['Neighborhood']

for ind in np.arange(parkwoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(parkwoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Hotel,Restaurant,Asian Restaurant,Gym,Bar
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Sandwich Place,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Bakery,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Coffee Shop,Pizza Place,Sandwich Place,Beer Store,Fast Food Restaurant,Gluten-free Restaurant,Dance Studio
4,"Alderwood,Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,General Travel
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fried Chicken Joint,Sandwich Place,Sushi Restaurant,Shopping Mall,Frozen Yogurt Shop,Bank,Restaurant,Diner,Bridal Shop
6,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Diner,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park,Lawrence Manor East",Fast Food Restaurant,Sushi Restaurant,Coffee Shop,Italian Restaurant,Pharmacy,Pizza Place,Café,Pub,Butcher,Restaurant
8,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Farmers Market,Italian Restaurant,Pub,Café,Cheese Shop,Steakhouse
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [46]:
# set number of clusters
kclusters = 5

parkwoods_grouped_clustering = parkwoods_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(parkwoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

In [47]:
neighborhoods_venues_sorted.shape

(99, 11)

In [48]:
kmeans.labels_

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 4, 3, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4,
       0, 4, 0, 0, 0, 0, 4, 0, 4, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4])

In [49]:
parkwoods_venues.shape

(2229, 7)

In [ ]:
parkwoods_merged = dff

# add clustering labels
parkwoods_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
parkwoods_merged = parkwoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

parkwoods_merged.head() 

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

We can examine each cluster and determine the discriminating venue categories that distinguish each cluster and assign names accordingly.

#### Cluster 1

In [ ]:
parkwoods_merged.loc[parkwoods_merged['Cluster Labels'] == 0, parkwoods_merged.columns[[1] + list(range(5, parkwoods_merged.shape[1]))]]

#### Cluster 2

In [ ]:
parkwoods_merged.loc[parkwoods_merged['Cluster Labels'] == 1, parkwoods_merged.columns[[1] + list(range(5, parkwoods_merged.shape[1]))]]

#### Cluster 3

In [ ]:
parkwoods_merged.loc[parkwoods_merged['Cluster Labels'] == 2, parkwoods_merged.columns[[1] + list(range(5, parkwoods_merged.shape[1]))]]

#### Cluster 4

In [ ]:
parkwoods_merged.loc[parkwoods_merged['Cluster Labels'] == 3, parkwoods_merged.columns[[1] + list(range(5, parkwoods_merged.shape[1]))]]

#### Cluster 5

In [ ]:
parkwoods_merged.loc[parkwoods_merged['Cluster Labels'] == 4, parkwoods_merged.columns[[1] + list(range(5, parkwoods_merged.shape[1]))]]